#### Exercise 5.1

Write a function to convert `LazyList` to a `List`, which will force its evaluatio and let you look at in the REPL. You can convert to the regular `List` type in the standard library, and you can place this and other functions that operate on a `LazyList` inside the `LazyList` enum.

#### Exercise 5.2

Write the function `take(n)` for returning the first `n` elements of a `LazyList` and `drop(n)` for skipping the first `n` elements of a `LazyList`. Define these functions inside `LazyList` enum.

#### Exercise 5.3

Write the function `takeWhile` for returning all starting elements of a `LazyList` that match the given predicate.

In [1]:
enum LazyList[+A]:
    case Empty
    case Cons(h: () => A, t: () => LazyList[A])

    def toList: List[A] = this match
        case Empty => Nil
        case Cons(h, t) => h() :: t().toList

    def take(n: Int): LazyList[A] = this match
        case Cons(h, t) if n > 0 => Cons(() => h(), () => t().take(n-1))
        case _ => LazyList.empty

    def drop(n: Int): LazyList[A] = this match
        case Cons(h, t) if n > 0 => t().drop(n-1)
        case Cons(h, t) => Cons(() => h(), () => t())
        case _ => LazyList.empty

    def takeWhile(p: A => Boolean): LazyList[A] = this match
        case Cons(h, t) if p(h()) => LazyList.cons(h(), t().takeWhile(p))
        case _ => LazyList.empty

object LazyList:
    def cons[A](
        hd: => A, tl: => LazyList[A]
    ): LazyList[A] =
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)

    def empty[A]: LazyList[A] = Empty

    def apply[A](as: A*): LazyList[A] =
        if as.isEmpty then empty
        else cons(as.head, apply(as.tail*))

defined class LazyList
defined object LazyList

The answers in the books give better implementation that are both more efficent and tail recursive.

In [2]:
enum LazyList[+A]:
    case Empty
    case Cons(h: () => A, t: () => LazyList[A])

    def toList: List[A] =
        @annotation.tailrec
        def go(ll: LazyList[A], acc: List[A]): List[A] = 
            ll match
                case Cons(h, t) => go(t(), h() :: acc)
                case Empty => acc.reverse
        go(this, Nil)

    def take(n: Int): LazyList[A] = this match
        case Cons(h, t) if n > 1 => LazyList.cons(h(), t().take(n-1))
        case Cons(h, _) if n == 1 => LazyList.cons(h(), LazyList.empty)
        case _ => LazyList.empty

    @annotation.tailrec
    final def drop(n: Int): LazyList[A] = this match
        case Cons(h, t) if n > 0 => t().drop(n-1)
        case _ => this

object LazyList:
    def cons[A](
        hd: => A, tl: => LazyList[A]
    ): LazyList[A] =
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)

    def empty[A]: LazyList[A] = Empty

    def apply[A](as: A*): LazyList[A] =
        if as.isEmpty then empty
        else cons(as.head, apply(as.tail*))

defined class LazyList
defined object LazyList

In [3]:
import LazyList.*

LazyList(1, 2, 3).take(2).toList

import LazyList.*


res2_1: List[Int] = List(1, 2)

In [4]:
extension [A](ll: LazyList[A])
    def foldRight[B](acc: => B)(f: (A, => B) => B): B =
        ll match
            case Cons(h, t) => f(h(), t().foldRight(acc)(f))
            case _ => acc

    def exists(p: A => Boolean): Boolean =
        ll.foldRight(false)((a: A, b) => p(a) || b)

LazyList(1, 2, 3).exists(_ > 0)

defined extension methods 
res3_1: Boolean = true

#### Exercise 5.4

Implement `forAll`, which checks that all elements in the `LazyList` match a given predicate. Your implementation should terminate the traversal as soon as it encounters a nonmatching value.

In [7]:
extension [A](ll: LazyList[A])
    def forAll(p: A => Boolean): Boolean =
        ll.foldRight(true)((a, b) => p(a) && b)

LazyList(2, 4, 6).forAll(_ % 2 == 0)

defined extension methods 
res6_1: Boolean = true

In [13]:
extension [A](ll: LazyList[A])
    def takeWhile(p: A => Boolean): LazyList[A] =
        ll.foldRight(empty)((a, b) => if p(a) then LazyList.cons(a, b) else empty)

defined extension methods 

#### Exercise 5.5

Use `foldRight` to implement `takeWhile`.

In [14]:
LazyList(1, 2, 3, 10, 11, 12).takeWhile(_ < 10).toList

res13: List[Int] = List(1, 2, 3)

#### Exercise 5.6

Implement `headOption` using `foldRight`.

In [16]:
extension [A](ll: LazyList[A])
    def headOption: Option[A] =
        ll.foldRight(None : Option[A])((a, b) => Some(a))

LazyList(1, 2).headOption
empty.headOption

defined extension methods 
res15_1: Option[Int] = Some(value = 1)
res15_2: Option[Nothing] = None

#### Exercise 5.7

Implement `map`, `filter`, `append`, and `flatMap` using `foldRight`. The `append` method should be nonstrict in its argument.

In [17]:
extension [A](ll: LazyList[A])
    def map[B](f: A => B): LazyList[B] =
        ll.foldRight(empty)((a, b) => cons(f(a), b))

LazyList(1, 2, 3).map(_ + 1).toList

defined extension methods 
res16_1: List[Int] = List(2, 3, 4)

In [29]:
extension [A](ll: LazyList[A])
    def filter(f: A => Boolean): LazyList[A] =
        ll.foldRight(empty)((a, b) => if f(a) then cons(a, b) else b)

LazyList(1, 2, 3).filter(_ % 2 == 0).toList

defined extension methods 
res28_1: List[Int] = List(2)

In [26]:
extension [A](ll: LazyList[A])
    def append(that: => LazyList[A]): LazyList[A] =
        ll.foldRight(that)((a, b) => cons(a, b))

LazyList(1, 2, 3).append(LazyList(4, 5, 6)).toList

defined extension methods 
res25_1: List[Int] = List(1, 2, 3, 4, 5, 6)

In [25]:
extension [A](ll: LazyList[A])
    def flatMap[B](f: A => LazyList[B]): LazyList[B] =
        ll.foldRight(empty)((a, b) => f(a).append(b))

def LazyRangeList(start: Int, end: Int): LazyList[Int] =
    if start > end then empty 
    else if start == end then cons(start, empty)
    else cons(start, LazyRangeList(start+1, end))

LazyRangeList(1, 4).flatMap(a => LazyRangeList(1, a)).toList

defined extension methods 
defined function LazyRangeList
res24_2: List[Int] = List(1, 1, 2, 1, 2, 3, 1, 2, 3, 4)

#### Exercise 5.8

Generalize `ones` slightly to the function `continually`, which returns an infinite `LazyList` of a given value.

In [30]:
def continually[A](a: A): LazyList[A] = cons(a, continually(a))

continually("foo").take(3).toList

defined function continually
res29_1: List[String] = List("foo", "foo", "foo")

#### Exercise 5.9

Write a function that generates an infinite lazy list of integers start from n, then n+1, ....

In [31]:
def from(n: Int): LazyList[Int] = cons(n, from(n+1))

from(1).take(10).toList

defined function from
res30_1: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

#### Exercise 5.10

Write a function `fibs` that generates the infinite lazy list of Fibonacci numbers: 0, 1, 1, 2, 3, 5, 8 and so on.

In [34]:
def fib : LazyList[Int] =
    def go(a: Int, b: Int): LazyList[Int] = cons(a, go(b, a + b))
    go(0, 1)

fib.take(7).toList

defined function fib
res33_1: List[Int] = List(0, 1, 1, 2, 3, 5, 8)

#### Exercise 5.11

Write a more general `LazyList`-building function called `unfold`. It takes an initial state and a function for producing both the next state and the next value in the generated lazy list. `Option` is used to indicate when the `LazyList` should be terminated, if at all. The function `unfold` is a very general `LazyList`-building function.

In [35]:
def unfold[A, S](state: S)(f: S => Option[(A, S)]): LazyList[A] =
    f(state) match
        case Some((a, s)) => cons(a, unfold(s)(f))
        case None => empty

defined function unfold

#### Exercise 5.12

Write `fibs`, `from`, `continually`, and `ones` in terms of `unfold`.

In [37]:
def fibs = unfold((0, 1))((curr, next) => Some((curr, (next, curr+next))))

fibs.take(7).toList

defined function fibs
res36_1: List[Int] = List(0, 1, 1, 2, 3, 5, 8)

In [38]:
def from(n: Int): LazyList[Int] = unfold(n)(n => Some(n, n+1))

from(10).take(5).toList

defined function from
res37_1: List[Int] = List(10, 11, 12, 13, 14)

In [41]:
def continually[A](a: A): LazyList[A] = unfold(())(_ => Some(a, ()))

def ones : LazyList[Int] = unfold(())(_ => Some(1, ()))

continually("foo").take(3).toList
ones.take(5).toList

defined function continually
defined function ones
res40_2: List[String] = List("foo", "foo", "foo")
res40_3: List[Int] = List(1, 1, 1, 1, 1)

#### Exercise 5.13

Use `unfold` to implement `map`, `take`, `takeWhile`, `zipWith` (as in chapter 3), and `zipAll`. The `zipAll` function should continue the traversal as long as either lazy list has more elements; it uses `Option` to indicate whether each lazy list has been exhausted.

In [46]:
extension [A](ll: LazyList[A])
    def map[B](f: A => B): LazyList[B] =
        unfold(ll)(_ match 
            case Cons(h, t) => Some(f(h()), t())
            case Empty => None)

LazyList(1, 2, 3).map(_ * 2).toList

defined extension methods 
res45_1: List[Int] = List(2, 4, 6)

In [51]:
extension [A](ll: LazyList[A])
    def take(n: Int): LazyList[A] =
        unfold((n, ll))((n, ll) => ll match 
            case Cons(h, t) if n > 1 => Some(h(), (n-1, t()))
            case Cons(h, _) if n == 1 => Some(h(), (n-1, empty))
            case _ => None
        )

    def takeWhile(p: A => Boolean): LazyList[A] =
        unfold(ll)(_ match 
            case Cons(h, t) if p(h()) => Some(h(), t())
            case _ => None
        )

    def zipWith[B](that: LazyList[B]): LazyList[(A, B)] =
        unfold((ll, that))(_ match
            case (Cons(h, t), Cons(h2, t2)) => Some((h(), h2()), (t(), t2()))
            case _ => None
        )

    def zipAll[B](that: LazyList[B]): LazyList[(Option[A], Option[B])] =
        unfold((ll, that))(_ match
            case (Cons(h, t), Cons(h2, t2)) => Some((Some(h()), Some(h2())), (t(), t2()))
            case (Cons(h, t), _) => Some((Some(h()), None), (t(), empty))
            case (_, Cons(h2, t2)) => Some((None, Some(h2())), (empty, t2()))
            case _ => None
        )

from(1).zipWith(from(2)).take(5).toList

LazyList(1, 2, 3).zipAll(from(1)).take(6).toList

defined extension methods 
res50_1: List[(Int, Int)] = List((1, 2), (2, 3), (3, 4), (4, 5), (5, 6))
res50_2: List[(Option[Int], Option[Int])] = List(
  (Some(value = 1), Some(value = 1)),
  (Some(value = 2), Some(value = 2)),
  (Some(value = 3), Some(value = 3)),
  (None, Some(value = 4)),
  (None, Some(value = 5)),
  (None, Some(value = 6))
)

#### Exercise 5.14

Implement `startsWith` using functions you've written. It should check if one `LazyList` is a prefix of another.

In [66]:
extension [A](ll: LazyList[A])
    def startsWith(prefix: LazyList[A]): Boolean =
        ll.zipAll(prefix).takeWhile(_(1).isDefined).forAll(_ == _)

LazyList(1, 2, 3).startsWith(LazyList(1, 2))
LazyList(1, 2, 3).startsWith(LazyList(1, 2, 3))
LazyList(1, 2, 3).startsWith(LazyList(1, 2, 3, 4))

defined extension methods 
res65_1: Boolean = true
res65_2: Boolean = true
res65_3: Boolean = false

#### Exercise 5.15

Implement `tails` using `unfold`. For a given `LazyList`, `tails` returns the `LazyList` of suffixes of the input sequence, starts with the original `LazyList`. For example, given `LazyList(1, 2, 3)`, it would return `LazyList(LazyList(1, 2, 3), LazyList(2, 3), LazyList(3), LazyList())`.

In [82]:
extension [A](ll: LazyList[A])
    def tails = unfold(ll)(_ match
        case ll@Cons(_, t) => Some(ll, t())
        case _ => None
    )

LazyList(1, 2, 3).tails.toList.map(_.toList)


defined extension methods 
res81_1: List[List[Int]] = List(List(1, 2, 3), List(2, 3), List(3))

#### Exercise 5.16

Generalize `tails` to the function `scanRight`, which is like a `foldRight` that returns a lazy list of the intermediate results. For example:

```scala
LazyList(1, 2, 3).scanRight(0)(_ + _).toList
```
Results in
`List(6, 5, 3, 0)`

In [86]:
extension [A](ll: LazyList[A])
    def scanRight[B](acc: B)(f: (A, B) => B): LazyList[B] =
        unfold((ll, acc))((ll, acc) => ll match
            case Cons(h, t) => Some(f(h(), acc), (t(), acc))
            case _ => None
        )

LazyList(1, 2, 3).scanRight(0)(_ + _).toList

defined extension methods 
res85_1: List[Int] = List(1, 2, 3)

That clearly did not work. What we need to do is add the list to itself somehow. Like (1, 2, 3) + (2, 3) + (3) + initAcc. Or we could foldRight and take the head of the accumulator and add it the current head.

In [88]:
extension [A](ll: LazyList[A])
    def scanRight[B](acc: B)(f: (A, B) => B): LazyList[B] =
        ll.foldRight(LazyList(acc))((a, acc) => cons(f(a, acc.headOption.get), acc))

LazyList(1, 2, 3).scanRight(0)(_ + _).toList

defined extension methods 
res87_1: List[Int] = List(6, 5, 3, 0)